<a href="https://colab.research.google.com/github/lucasPDY/Show-Me-Your-PPE/blob/master/object_detection_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### **Inroduction**:


Aiming to minimize police response time by detecting weapons in a live cctv camera. The main motivation of this project is due to the increasing number of school mass shootings in the U.S.


### This notebook is a part of this [medium post](https://medium.com/@alaasinjab/detailed-tutorial-build-your-custom-real-time-object-detector-5ade1017fd2d).

### This notebook was designed to be ran from top to bottom without the need to mount Google Drive

## PPE Detection Using Tensorflow Object Detection API

Workspace structure


```
# This is formatted as code
```


```
ppe_detection/
        ├─ data/
        │    ├── images/
        │    │      ├── armas (1).jpg
        │    │      ├── armas (2).jpg
        │    │      └── ...
        │    ├── train_labels/
        │    │      ├── armas (1).xml
        │    │      ├── armas (2).xml
        │    │      └── ...
        │    ├── test_labels/
        │    │      ├── armas (10).xml
        │    │      ├── armas (20).xml
        │    │      └── ...
        │    ├── label_map.pbtxt
        │    ├── test_labels.csv
        │    ├── train_labels.csv
        │    ├── test_labels.record
        │    └── train_labels.record
        └─ models/
             ├─ research/
             │      ├── fine_tuned_model/
             │      │         ├── frozen_inference_graph.pb
             │      │         └── ...
             │      │         
             │      ├── pretrained_model/
             │      │         ├── frozen_inference_graph.pb
             │      │         └── ...
             │      │         
             │      ├── object_detection/
             │      │         ├── utils/
             │      │         ├── samples/
             │      │         │      ├── samples/ 
             │      │         │      │       ├── configs/             
             │      │         │      │       │     ├── ssd_mobilenet_v2_coco.config
             │      │         │      │       │     ├── rfcn_resnet101_pets.config
             │      │         │      │       │     └── ...
             │      │         │      │       └── ... 
             │      │         │      └── ...                                
             │      │         ├── export_inference_graph.py
             │      │         ├── model_main.py
             │      │         └── ...
             │      │         
             │      ├── training/
             │      │         ├── events.out.tfevents.xxxxx
             │      │         └── ...               
             │      └── ...
             └── ...




## Choosing a pre training model
The model used for this project is `ssd_mobilenet_v2_coco`.
Check other models from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

Because the interestes of this project is to interfere on real time video, i am chosing a model that has a high inference speed `(ms)` with relativly high `mAP` on COCO

In [0]:


# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
    }
}

# Select a model in `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'


## Installing Required Packages 

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

Selecting previously unselected package python-bs4.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [0]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [0]:
#we need tenorflow v 1.15.0, object detection API is removed from tf v 2.0+
print(tf.__version__)

1.15.2


In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Downloading and Orgniazing Images and Annotations
1. Downloading the images and annotations from the [source](https://sci2s.ugr.es/weapons-detection)  and unziping them
2. Creating a directory `(data)` to save some data such as; images, annotation, csv, etc...
3. Creating two directories; for the training and testing labels (not the images)
4. Randomly splitting our labels into 80% training and 20% testing and moving the splits to their directories: `(train_labels)` & `(test_labels)` 

In [0]:
#creates a directory for the whole project
!mkdir ppe_detection

mkdir: cannot create directory ‘ppe_detection’: File exists


In [0]:
%cd ppe_detection
!mkdir data

/content/ppe_detection


In [0]:
#upload the data 

In [0]:

!unzip -q "/content/drive/Shared drives/FIT3161_2019S2_Team08/FIT3162/Data/new data/new 3.zip" -d "/content/ppe_detection/data/"


In [0]:

# moves the annotations to data folder
# !mv data/test_labels/* data/train_labels
%cd /content/ppe_detection
!mv data/"new 3"/* data/


/content/ppe_detection


In [0]:
!rm -rf /content/ppe_detection/data/'new 3'

In [0]:

# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 540 labels to the testing dir: `test_labels`
!ls data/train_labels/* | sort -R | head -540 | xargs -I{} mv {} data/test_labels

In [0]:
# 2400 "images"(xml) for training
ls -1 data/train_labels/ | wc -l

5309


In [0]:
# 600 "images"(xml) for testing
ls -1 data/test_labels/ | wc -l

SyntaxError: ignored

In [0]:
%ls
!ls -1 data/images/ | wc -l

data/
8093


## Preprocessing Images and Labels
1. Converting the annotations from xml files to two csv files for each `train_labels/` and `train_labels/`.
2. Creating a pbtxt file that specifies the number of class (one class in this case)
3. Checking if the annotations for each object are placed within the range of the image width and height.

In [0]:

#adjusted from: https://github.com/datitran/raccoon_dataset

#converts the annotations/labels into one csv file for each training and testing labels
#creats label_map.pbtxt file

%cd /content/ppe_detection/data/

# images extension
images_extension = 'jpg'
# takes the path of a directory that contains xml files and converts
#  them to one csv file.

# returns a csv file that contains: image name, width, height, class, xmin, ymin, xmax, ymax.
# note: if the xml file contains more than one box/label, it will create more than one row for the same image. each row contains the info for an individual box. 
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    try:
      tree = ET.parse(xml_file)
      root = tree.getroot()
    except: 
      print("error ")
      continue

    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)

  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

#creats a pbtxt file the has the class names.
# classes=['hardhat','person','vest']
print(classes)
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: '{1}'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)


/content/ppe_detection/data
Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
['hardhat', 'person', 'vest']


In [0]:
#checking the pbtxt file
!cat label_map.pbtxt

item {
    id: 1
    name: 'hardhat'
    display_name: 'hardhat'
 }

item {
    id: 2
    name: 'person'
    display_name: 'person'
 }

item {
    id: 3
    name: 'vest'
    display_name: 'vest'
 }

In [0]:
# they are there!
ls -l

total 524
drwxrwxr-x 2 root root 114688 Apr 26 05:21 images/
-rw-r--r-- 1 root root    197 Apr 26 05:28 label_map.pbtxt
drwxr-xr-x 3 root root   4096 Apr 26 05:27 __MACOSX/
drwxr-xr-x 2 root root  28672 Apr 26 05:22 test_labels/
-rw-r--r-- 1 root root  52427 Apr 26 05:28 test_labels.csv
drwxr-xr-x 2 root root  73728 Apr 26 05:22 train_labels/
-rw-r--r-- 1 root root 244397 Apr 26 05:28 train_labels.csv


In [0]:
#checks if the images box position is placed within the image.

#note: while this doesn't checks if the boxes/annotatoins are correctly
# placed around the object, Tensorflow will through an error if this occured.
%cd /content/ppe_detection/data
# path to images
images_path = 'images'
import csv

errorDf = []
#loops over both train_labels and test_labels csv files to do the check
# returns the image name where an error is found 
# return the incorrect attributes; xmin, ymin, xmax, ymax.
for CSV_FILE in ['train_labels.csv','test_labels.csv']:
  with open(CSV_FILE, 'r') as fid:  
      print('[*] Checking file:', CSV_FILE) 
      file = csv.reader(fid, delimiter=',')
      first = True 
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          temp = {}
          if error == True:
              error_cnt += 1
              error = False         
          if first == True:
              first = False
              continue     
          cnt += 1      
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])     
          path = os.path.join(images_path, name)
          img = cv2.imread(path) 
          if type(img) == type(None):
              error = True
              print('Could not read image', img,path)
              continue     
          org_height, org_width = img.shape[:2]     
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)     
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height) 
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)  
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
          if error:
              # os.remove(path)
              temp['path'] = path
              errorDf.append(temp)

          
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")
errorDf= pd.DataFrame(errorDf)


/content/ppe_detection/data
[*] Checking file: train_labels.csv


KeyboardInterrupt: ignored

In [0]:
#removing the entry for it in the csv for that image as well

#because we did a random split for the data, we dont know if it ended up being in training or testing
# we will remove the image from both.

#training
#reading the training csv
df = pd.read_csv('/content/ppe_detection/data/train_labels.csv')
bad=["64b.jpg","crop_000011.jpg"]
# bad = ["person53.jpg", "person58.jpg", "person214.jpg", "person61.jpg", "person250.jpg", "person60.jpg", "person357.jpg", 
      #  "person1132.jpg", "person248.jpg", "person185.jpg","131.giving-more-than-money-inline1-1290374-2.png","134.michael-emerson.jpeg","133.melanie-person.jpg","13.maxresdefault.jpg"]
# removing armas (2815).jpg
df = df[~df['filename'].isin(bad)]
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/content/ppe_detection/data/train_labels.csv')


#testing
#reading the testing csv
df = pd.read_csv('/content/ppe_detection/data/test_labels.csv')
# removing armas (2815).jpg
df = df[~df['filename'].isin(bad)]
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/content/ppe_detection/data/test_labels.csv')

# Just for the memory
df = None


## Downloading the Tensorflow model API
1. Get the TensorFlow training model API by Cloning [Tensorflow Pre-train models](https://github.com/tensorflow/models.git) from the offical git repo. 
2. Compiling the protos and adding folders to the os environment.
3. Testing the model builder by running model_builder_test.py (tensorflow API).


In [0]:
# Downlaods Tenorflow model API
%cd /content/ppe_detection/
!git clone --q https://github.com/tensorflow/models.git

/content/ppe_detection


In [0]:
%cd /content/ppe_detection/models/research
#compiling the proto buffers (Protocol buffers are Google's language-neutral, platform-neutral, extensible mechanism for serializing structured data)
# Protocol buffers is kind of XML format but smaller,lighter and simpler.(https://developers.google.com/protocol-buffers/)
!protoc object_detection/protos/*.proto --python_out=.

# exports the PYTHONPATH environment variable with the reasearch and slim folders' paths
os.environ['PYTHONPATH'] += ':/content/ppe_detection/models/research/:/content/ppe_detection/models/research/slim/'

/content/ppe_detection/models/research
object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [0]:
# testing the model builder
# the purpose of doing this is to make sure the model train file is working 
!python3 object_detection/builders/model_builder_test.py

## Generating Tf record
- Generating two TFRecords files for the training and testing CSVs.
- Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage. Instead of loading the full data into memory, Tenorflow breaks the data into batches using these TFRecords automatically

In [0]:
#adjusted from: https://www.tensorflow.org/tutorials/load_data/tfrecord
#adjusted from: https://github.com/datitran/raccoon_dataset

# converts the csv files for training and testing data to two TFRecords files.
# places the output in the same directory as the input


from object_detection.utils import dataset_util
%cd /content/ppe_detection/models/

DATA_BASE_PATH = '/content/ppe_detection/data/'
image_dir = DATA_BASE_PATH +'images/'


#the row label "string" and return "id" have to be similar to the label_map.pbtxt that created earlier
def class_text_to_int(row_label):
		if row_label == 'hardhat':
				return 1
		elif row_label == 'person':
				return 2
		elif row_label == 'vest':
				return 3
		else:
				return 0


# Spilt the data in csv file by the data filename and object labeled in the data
def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


#creating the tf record example using the train and test labels csv file
#the the csv file it will extract the information of the image (height,weight,xymin/max)
def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels','test_labels']:
		with tf.io.TFRecordWriter(DATA_BASE_PATH+csv+'.record') as writer:
				path=os.path.join(image_dir)
				examples = pd.read_csv(DATA_BASE_PATH + csv+'.csv')
				grouped = split(examples,'filename')
				for group in grouped:
						try:
								tf_example = create_tf_example(group,path)
								writer.write(tf_example.SerializeToString())
						except:
								continue

		output_path = os.path.join(os.getcwd(),DATA_BASE_PATH+csv+'.record')
		print("Successfully created TFRecord: {}".format(DATA_BASE_PATH + csv + '.record'))



/content/ppe_detection/models
Successfully created TFRecord: /content/ppe_detection/data/train_labels.record
Successfully created TFRecord: /content/ppe_detection/data/test_labels.record


## Downloading the Pre-train tensorflow Model
1. Based on the model selecting to be fine tune at the top of this notebook, downloading the model selected and extracting its content.
2. Creating a dir to save the model while training.

In [0]:
%cd /content/ppe_detection/models/research

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#the distination folder where the model will be saved
fine_tune_dir = '/content/ppe_detection/models/research/pretrained_model'

#checks if the model has already been downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the file and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

/content/ppe_detection/models/research


In [0]:
#checking the content of the pretrained model.
# this is the directory of the "fine_tune_checkpoint" that is used in the config file.
!echo {fine_tune_dir}
!ls -alh {fine_tune_dir}

/content/ppe_detection/models/research/pretrained_model
total 135M
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 .
drwxr-xr-x 63 root   root  4.0K May 18 06:12 ..
-rw-r--r--  1 345018 89939   77 Mar 30  2018 checkpoint
-rw-r--r--  1 345018 89939  67M Mar 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 89939  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 89939  15K Mar 30  2018 model.ckpt.index
-rw-r--r--  1 345018 89939 3.4M Mar 30  2018 model.ckpt.meta
-rw-r--r--  1 345018 89939 4.2K Mar 30  2018 pipeline.config
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 saved_model


## Configuring the Training Pipeline
1. Adding the path for the TFRecords files and pbtxt,batch_size,num_steps,num_classes to the configuration file.
2. Adding some Image augmentation for more data and visualise the data in differnt environment.
3. Creating a directory to save the model at each checkpoint while training. 

In [0]:

#the path to the folder containing all the sample config files
#in here we use back the configs file of the pre-train model and changing/adding the hyper parameter
CONFIG_BASE = "/content/ppe_detection/models/research/object_detection/samples/configs/"

#path to the specified model's config file
model_pipline = os.path.join(CONFIG_BASE, pipeline_file)
model_pipline

'/content/ppe_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'

In [0]:
#check the sample config file of the pre-train tf model
!cat /content/ppe_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 90
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect

In [0]:
#editing the configuration file to add the path for the TFRecords files, pbtxt,batch_size,num_steps,num_classes.
# any image augmentation, hyperparemeter tunning (drop out, batch normalization... etc) would be editted here

%%writefile {model_pipline}
model {
  ssd {
    num_classes: 3 # number of classes to be detected
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher { #the theshold of match and unmatch while training the model
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator { #using default calculate method
      iou_similarity { #intersection of union
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    # all images will be resized to the below W x H.
    image_resizer { 
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }

    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        use_dropout: true  # to counter over fitting. you can also try tweaking its probability below
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6, # Rectified Linear Unit Activation Function, it is output of that node given an input or set of inputs. if 1 activate ,else(0) no activate
          regularizer { #Regularization is a technique which makes slight modifications to the learning algorithm such that the model generalizes better
            l2_regularizer {
            weight: 0.0010 # higher regularizition to counter overfitting
          }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {  # Batch normalization is a technique for improving the speed, performance, and stability of artificial neural networks. Introduct in 2015 research paper
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor { 
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.001 # higher regularizition to counter overfitting
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000 
        iou_threshold: 0.99 #Intersection over Union  match is found if both boxes have an IoU greater or equal than 0.99.
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.65
        
        #adjust this to the max number of objects per class. 
        #there are some images with more than one up to 16.
        max_detections_per_class: 100
        # max number of detections among all classes
        max_total_detections: 100
      }
      score_converter: SIGMOID #activation function algorithm 
    }
  }
}

######below are model training configuration#########
train_config: { 
  batch_size: 18 # amount of samples you feed in your network 
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }

  #the path to the pretrained model. 
  fine_tune_checkpoint: "/content/ppe_detection/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  from_detection_checkpoint:true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  # num_steps: 200000 
  

  #data augmentaion is done here, you can remove or add more.
  # They will help the model generalize but the training time will increase greatly by using more data augmentation.
  # Check this link to add more image augmentation: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto
  
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    #path to the training TFRecord
    input_path: "/content/ppe_detection/data/train_labels.record"
  }
  #path to the label map 
  label_map_path: "/content/ppe_detection/data/label_map.pbtxt"
}

eval_config: {
  # the number of images in your "testing" data 
  num_examples: 1392
  # the number of images to disply in Tensorboard while training
  num_visualizations: 20

  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  # max_evals: 10
  # max_evals: 20

  use_moving_averages: false
}

eval_input_reader: {
  tf_record_input_reader {
      
    #path to the testing TFRecord
    input_path: "/content/ppe_detection/data/test_labels.record"
  }
  #path to the label map 
  label_map_path: "/content/ppe_detection/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content/ppe_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config


In [0]:
#!!!!! do not execute this if you model has done some training process

# where the model will be saved at each checkpoint while training 
model_dir = 'training/'

# remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Tensorboard
1. Downlaoding and unzipping Tensorboard
2. creating a link to visualize multiple graph while training.


notes: 
  1. Tensorboard will not log any files until the training starts. 
  2. a max of 20 connection per minute is allowed when using ngrok, you will not be able to access tensorboard while the model is logging.

In [0]:
#downlaoding ngrok to be able to access tensorboard on google colab
#using ngrok we can generate a public access link to access the tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-05-18 04:01:09--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.54.251.217, 52.22.244.167, 52.44.6.119, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.54.251.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.0MB/s    in 0.4s    

2020-05-18 04:01:10 (37.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
#the logs that are created while training 
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
#The link to tensorboard.
#works after the training starts.

### note: if you didnt get a link as output, rerun this cell and the one above
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://93b7c589.ngrok.io


## Training

In here, we start to train the model.
Observe tensorboard to change the hyperparameter (weight,match/unmatch threshold)


In [0]:

!python3 /content/ppe_detection/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipline}\
    --model_dir={model_dir} \
    --alsologtostderr \

#alsologtostderr: it sends the logs to STDERR standard file, which would allow you to append at the end of the command



W0518 07:09:01.627266 140019324786560 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/utils/config_util.py:137: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0518 07:09:01.631516 140019324786560 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/model_lib.py:685: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0518 07:09:01.631685 140019324786560 model_lib.py:686] Forced number of epochs for all eval validations to be 1.

W0518 07:09:01.631893 140019324786560 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/utils/config_util.py:523: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Maybe overwriting train_steps: None
I0518 07:09:01.632059 140019324786560 config_util.py:523] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bflo

## Exporting The Trained model



In [0]:
!rm -rf /content/ppe_detection/models/research/fine_tuned_model/saved_model


In [0]:


#the location where the exported model will be saved in.
output_directory = '/content/ppe_detection/models/research/fine_tuned_model'

# goes through the model is the training/ dir and gets the last one.
# you could choose a specfic one instead of the last
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

#exports the model specifed and inference graph
!python /content/ppe_detection/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_pipline} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

training/model.ckpt-2227


W0518 07:23:47.098869 140375051868032 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0518 07:23:47.106734 140375051868032 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/exporter.py:419: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0518 07:23:47.106997 140375051868032 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/exporter.py:138: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0518 07:23:47.147287 140375051868032 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/core/preprocessor.py:3030: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.


W0518 07:23:47.180166 140375051868032 module_wrapper.py:13

In [0]:
#downloads the frozen model that is needed for inference
files.download(output_directory + '/frozen_inference_graph.pb')

In [0]:
#downlaod the for converting to TF.js (web freindly format)
files.download(output_directory + '/saved_model/saved_model.pb')


In [0]:
#downlaod the label map
files.download(DATA_BASE_PATH + '/label_map.pbtxt')

### **Model Accuracy**

In [0]:
# use the trained model and the test data to get the predict result by your model
!python /content/ppe_detection/models/research/object_detection/inference/infer_detections.py --input_tfrecord_paths=/content/ppe_detection/data/test_labels.record --output_tfrecord_path=/content/ppe_detection/data/detections.record --inference_graph=/content/ppe_detection/models/research/fine_tuned_model/frozen_inference_graph.pb



W0518 07:24:08.016309 140467443513216 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/inference/infer_detections.py:57: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0518 07:24:08.016567 140467443513216 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/inference/infer_detections.py:57: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0518 07:24:08.016789 140467443513216 module_wrapper.py:139] From /content/ppe_detection/models/research/object_detection/inference/infer_detections.py:65: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-05-18 07:24:08.018596: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-18 07:24:08.033448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from

In [0]:
%cd /content/ppe_detection/data
!git clone https://github.com/svpino/tf_object_detection_cm


Cloning into 'tf_object_detection_cm'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 31 (delta 1), reused 1 (delta 0), pack-reused 21
Unpacking objects: 100% (31/31), done.


In [0]:
# create the confusion matrix 
%cd /content/ppe_detection/data/tf_object_detection_cm/
!python confusion_matrix.py --detections_record=/content/ppe_detection/data/detections.record --label_map=/content/ppe_detection/data/label_map.pbtxt --output_path=/content/ppe_detection/data/confusion_matrix.csv


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0518 08:30:03.986804 140188830357376 deprecation.py:323] From confusion_matrix.py:39: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Processed 100 images
Processed 200 images
Processed 300 images
Processed 400 images
Processed 500 images
Processed 600 images
Processed 700 images
Processed 800 images
Processed 900 images
Processed 1000 images
Processed 1100 images
Processed 1200 images
Processed 1300 images
Processed 1392 images

Confusion Matrix:
[[1.190e+03 1.190e+02 5.500e+01 7.240e+02]
 [1.000e+01 1.215e+03 1.000e+00 2.090e+02]
 [6.000e+00 3.000e+00 1.630e+02 1.800e+01]
 [1.530e+02 2.580e+02 7.600e+01 0.000e+00]] 

  category  ...  recall_@0.0IOU
0  hardhat  ...        0.569923
1   person  ...        0.846690
2     vest  ...        0.857895

[3 r